In [1]:
import pandas as pd
import os

In [30]:
root_path = r'C:\Users\ADE17\Desktop\Masters\Projects\AIChallenge_OWL\SollIch-Hackathon_Daten\Data_Participants3'

In [92]:
def load_data(root_path, mode='train'):
    data_dict = {}
    if mode == 'train':
        x_folder = "Train_X"
        y_folder = "Train_Y"
    else:
        x_folder = "Eval_X"
        y_folder = "Eval_Y"
    x_files = os.listdir(os.path.join(root_path, x_folder))
    y_files = os.listdir(os.path.join(root_path, y_folder))

    for x_file, y_file in zip(x_files, y_files):
        if x_file.endswith('.pq') and y_file.endswith('.pq'):
            path_X = os.path.join(x_folder, x_file)
            path_Y = os.path.join(y_folder, y_file)
            df_X = pd.read_parquet(os.path.join(root_path, path_X))
            df_Y = pd.read_parquet(os.path.join(root_path, path_Y))
            target_col_1 = df_X['ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage']
            targets = pd.concat([df_Y, target_col_1], axis=1)
            df_X = df_X.drop(['ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage'], axis=1)
            date = x_file.split('_')[1:]  # Extracting month and day
            date_key = '_'.join(date)[:5]  # Creating the 'MM_DD' format
            data_dict[date_key] = {'features': df_X, 'targets': targets}

    return data_dict

In [95]:
train_data = load_data(root_path)
val_data = load_data(root_path, mode='val')

In [184]:
import numpy as np
def custom_weighted_error(true_values, predicted_values):
    absolute_errors = np.abs(true_values - predicted_values)
    
    points = 0
    for error in absolute_errors:
        if error <= 0.05:
            points += 1
        elif 0.05 < error <= 0.1:
            points += 0.5
        elif 0.1 < error <= 0.5:
            points += 0.25
        else:
            points += 0
    
    return points 

In [148]:
def add_time_columns(df):
    df['Hour'] = df.index.hour
    df['Minute'] = df.index.minute
    df['Second'] = df.index.second
    return df

In [173]:
sample_train_features_1 = add_time_columns(train_data['03_14']['features'])
sample_train_targets_1 = train_data['03_14']['targets']
sample_train_features_2 = add_time_columns(train_data['03_15']['features'])
sample_train_targets_2 = train_data['03_15']['targets']
sample_val_features = add_time_columns(val_data['03_16']['features'])
sample_val_targets = val_data['03_16']['targets']

In [176]:
final_train_feat = pd.concat([sample_train_features_1, sample_train_features_2], axis=0)
final_train_targets = pd.concat([sample_train_targets_1, sample_train_targets_2], axis=0)

In [181]:
print(final_train_feat.shape)
print(final_train_targets.shape)
print(sample_val_features.shape)
print(sample_val_targets.shape)

(136802, 16)
(136802, 2)
(68401, 16)
(68401, 2)


In [174]:
from supervised.automl import AutoML

In [182]:
baseline_model = AutoML(mode='Compete', explain_level=2, ml_task='regression', eval_metric='mae', 
                        start_random_models=5)

In [185]:
baseline_model.fit(final_train_feat, final_train_targets)

Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 0.072804 trained in 21.48 seconds
Disable stacking for split validation
* Step simple_algorithms will try to check up to 2 models
2_DecisionTree mae 0.041563 trained in 2.78 seconds
3_DecisionTree mae 0.041563 trained in 2.83 seconds
* Step default_algorithms will try to check up to 6 models
4_Default_LightGBM mae 0.0046

AutoML(eval_metric='mae', explain_level=2, ml_task='regression', mode='Compete',
       start_random_models=5)

In [186]:
preds = baseline_model.predict(sample_val_features)

In [188]:
preds.shape

(68401,)

In [193]:
sample_val_targets['ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage'].shape

(68401,)

In [202]:
sample_val_features = add_time_columns(val_data['03_30']['features'])
sample_val_targets = val_data['03_30']['targets']

preds = baseline_model.predict(sample_val_features)
custom_weighted_error(sample_val_targets['ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage'],
                      preds) / len(preds) * 100

85.3459744740574